# LLM-based event detection

In [1]:
# pip install openai

In [2]:
# pip install fastapi-poe

In [3]:
import os
from dotenv import load_dotenv
import openai
import pandas as pd

from src.vars import OUTPUT_DIR, ROOT_DIR
import json
import re
import time

In [19]:
EXECUTE_LLM = False  # to do not waste resources by mistake
EXECUTE_PRELIMINARY_PHASE = False
EXTRACT_PRELIMINARY_SAMPLE = False

In [20]:
df = pd.read_csv(f'{OUTPUT_DIR}/df_complementary_info_filter_dates_keywords_acronims.csv')

In [21]:
if os.path.exists(f'{OUTPUT_DIR}/llm_output.csv'):
    df_llm_output = pd.read_csv(f'{OUTPUT_DIR}/df_llm_output.csv')
else:
    df_llm_output = pd.DataFrame(columns=['case_id', 'output'])
df_llm_output

,case_id,output


In [22]:
def extract_json_objects(text):
    # Pattern to find potential JSON objects (objects and arrays)
    array_pattern = r'(\[[^\[\]]*(\[[^\[\]]*\][^\[\]]*)*\])'

    # Find all potential matches
    assert type(text) == str
    potential_arrays = re.findall(array_pattern, text)

    # Extract just the matched strings
    potential_jsons = [match[0] for match in potential_arrays]

    # Try to parse each potential match
    valid_jsons = []

    for json_str in potential_jsons:
        try:
            # Check if it's valid JSON
            json_obj = json.loads(json_str)
            valid_jsons.append(json_str)
        except json.JSONDecodeError:
            # Skip invalid JSON
            continue

    return [json.dumps(json.loads(obj)) for obj in valid_jsons]

In [23]:
sys_prompt = ''
with open(f'{ROOT_DIR}/res/prompts/System Prompt.txt', encoding='utf-8') as file:
    sys_prompt = file.read()

In [24]:
if EXTRACT_PRELIMINARY_SAMPLE:
    idx_sample = df.sample(n=10).case_id
    idx_sample.to_csv(f'{OUTPUT_DIR}/preliminary_evaluation/idx_sample.csv', index=False)
    df = df[df.case_id.isin(idx_sample)]
elif EXECUTE_PRELIMINARY_PHASE:
    idx_sample = pd.read_csv(f'{OUTPUT_DIR}/preliminary_evaluation/idx_sample.csv').case_id
    df = df[df.case_id.isin(idx_sample)]

In [25]:
df

,case_id,complementary_info
667,223-638730,Codice CIG: 922420351E (Deliberazione dell’aut...
928,021-051428,Si rinvia al Disciplinare. Ai sensi dell‘art. ...
1062,248-716991,1) Si procederà all'aggiudicazione anche in pr...
1241,067-177404,Si precisa che il termine perentorio di presen...
1334,235-678939,LA PRESENTE PROCEDURA SI È SVOLTA ATTRAVERSO U...
1512,147-421403,Decreto dirigenziale di aggiudicazione efficac...
1567,209-597746,La gara è identificata con CIG dell’Autorità N...
2148,070-185352,La procedura viene svolta attraverso la piatta...
2553,105-294339,INFORMAZIONI RELATIVE AI LOTTI (segue dal punt...
2612,145-415881,Il disciplinare di gara è stato redatto nel ri...


In [11]:
def run_prompt(user_message: str, model: str):
    # time.sleep(3)
    result = dict()

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": user_message}],
        stream=True
    )
    result[model] = ''
    for chunk in response:
        if chunk.choices[0].delta.content:
            result[model] += chunk.choices[0].delta.content
    return result

def analyse_instances(df, model, idx):
    if EXECUTE_PRELIMINARY_PHASE:
        df_llm_output = pd.DataFrame(columns=['case_id', 'output'])
    executed_caseid = df_llm_output['case_id'].tolist()

    for index, row in df.iterrows():
        if row.case_id in executed_caseid:
            continue
        print(f"Row id: {index}")

        user_message = prompt % (df[df['case_id'] == row.case_id].iloc[0]['complementary_info'])
        new_row = dict()
        result = run_prompt(user_message, model)

        json_objs = extract_json_objects(result[model])
        if len(json_objs) == 1:
            json_objects = json_objs[0]
        elif len(json_objs) == 0:
            result = run_prompt(user_message, model)
            json_objs = extract_json_objects(result[model])
            if len(json_objs) == 1:
                json_objects = json_objs[0]
            elif len(json_objs) == 0:
                print(f"Error: no json objects found again in instance {index} -> '{row}'")
                continue

        time.sleep(5)
        new_row = {"case_id": row.case_id, "output": json_objs}
        df_llm_output.loc[len(df_llm_output)] = new_row

        if index % 5 == 0:
            if EXECUTE_PRELIMINARY_PHASE:
                df_llm_output.to_csv(f'{OUTPUT_DIR}/preliminary_evaluation/prompt_{idx}_{model}.csv', index=False)
            else:
                df_llm_output.to_csv(f'{OUTPUT_DIR}/llm_output.csv', index=False)

## Prompt validation and model selection

In [12]:
prompts = ["", "", ""]
with open(f'{ROOT_DIR}/res/prompts/prompt1 - zero shot.txt', encoding='utf-8') as file:
    prompts[0] = file.read()

In [13]:
prompts[1] = ""
with open(f'{ROOT_DIR}/res/prompts/prompt2 - few shot.txt', encoding='utf-8') as file:
    prompts[1] = file.read()

In [14]:
prompts[2] = ""
with open(f'{ROOT_DIR}/res/prompts/prompt3 - chain of thoughts.txt', encoding='utf-8') as file:
    prompts[2] = file.read()

In [15]:
if EXECUTE_LLM:
    load_dotenv()
    client = openai.OpenAI(
      api_key=os.getenv('POE_API_KEY'),
      base_url="https://api.poe.com/v1",
    )

    if EXECUTE_PRELIMINARY_PHASE:
        models = ["Llama-3.1-8B", "GPT-4o", "Claude-Sonnet-4"]
    else:
        models = ["Claude-Sonnet-4"]

    for idx, prompt in enumerate(prompts):
        for model in models:
            print(f"{model} - prompt {idx}")
            analyse_instances(df, model, idx)

In [16]:
df_llm_output

,case_id,output


In [1]:
print("Correct instances (manual annotation):")

print("\nPrompt 0 - Claude-Sonnet-4: 8/10")
print("Prompt 0 - GPT-4o: 7/10")
print("Prompt 0 - Llama-3.1-8B: 2/10")

print("\nPrompt 1 - Claude-Sonnet-4: 7/10")
print("Prompt 1 - GPT-4o: 7/10")
print("Prompt 1 - Llama-3.1-8B: 6/10")

print("\nPrompt 2 - Claude-Sonnet-4: 9/10")
print("Prompt 2 - GPT-4o: 8/10")
print("Prompt 2 - Llama-3.1-8B: 4/10")

Correct instances:

Prompt 0 - Claude-Sonnet-4: 8/10
Prompt 0 - GPT-4o: 7/10
Prompt 0 - Llama-3.1-8B: 2/10

Prompt 1 - Claude-Sonnet-4: 7/10
Prompt 1 - GPT-4o: 7/10
Prompt 1 - Llama-3.1-8B: 6/10

Prompt 2 - Claude-Sonnet-4: 9/10
Prompt 2 - GPT-4o: 8/10
Prompt 2 - Llama-3.1-8B: 4/10


In [18]:
# todo:
#  - esecuzione completa